# FRF_Currituck Sound Wave Data

Reading the FRF Thredds Server netCDF data for waves recorded in the Currituck Sound and writing these data, or a subset thereof, to a comma-separated values file. 

What data?

- time (converted to ISO 8601 format)
- waveHs  (significant wave height in meters)
- wavePeakFrequency (in s**-1)
- waveEnergyDensity (m**2 s)
- waveFrequency (s**-1)
- depth (water depth in m)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import netCDF4

%matplotlib inline

### Load source file location information and load copy of data to local container:

In [59]:
url_base='https://chlthredds.erdc.dren.mil/thredds/dodsC/frf/oceanography/waves/'
url='CS05-SBE26/CS05-SBE26.ncml'
urls=[
    'CS01-SBE26/CS01-SBE26.ncml',
    'CS02-SBE26/CS02-SBE26.ncml',
    'CS03-SBE26/CS03-SBE26.ncml',
    'CS04-SBE26/CS04-SBE26.ncml',
    'CS05-SBE26/CS05-SBE26.ncml']
    
nc=netCDF4.Dataset(url_base+url)

### Validate Variables:

Check for variables present in the data, along with their length. If the lengths differ (indicating missing data) it's probably best to just drop the variable from further consideration. If you really need it, you might be able to pad it to match the size of the other vars...

In [60]:
print('Variables present in the file, and their length (based on np.shape:')
for var in nc.variables:
    print(var, np.shape(nc.variables[var]))


Variables present in the file, and their length (based on np.shape:
latitude ()
longitude ()
station_name (64,)
waveFrequency (167,)
time (34165,)
waveHs (34165,)
wavePeakFrequency (34165,)
depth (34165,)
qcFlagE (34165,)
waveEnergyDensity (34165, 167)


### List the Station Position (Latitude and Longitude):

In [61]:
print(nc.variables['longitude'][0], nc.variables['latitude'][0])

-75.767118 36.178999


### Copy Data to Comma-Separated Values file:

In [62]:
times=nc.variables['time']

csdict={
'waveHs' : nc.variables['waveHs'],
'depth' : nc.variables['depth'],
'wavePeakFrequency' : nc.variables['wavePeakFrequency']
}

dt = netCDF4.num2date(times[:],times.units)
df = pd.DataFrame(csdict,index=dt)
df.to_csv('./FRF_'+url[11:-5]+'_Waves_ALL.csv')

/Applications/miniconda3/lib/python3.5/site-packages/pandas/core/dtypes/cast.py:866: UserWarning: Warning: converting a masked element to nan.
  v = np.array(v, copy=False)


## Parse wave data gathered by station into individual files by year and month...

In [3]:
file_path='/Users/paulp/GoogleDrive/projects/CurrituckSnd/CS_Stations/'
stations=['FRF_CS01-SBE26_Waves_ALL.csv','FRF_CS02-SBE26_Waves_ALL.csv','FRF_CS03-SBE26_Waves_ALL.csv',
         'FRF_CS04-SBE26_Waves_ALL.csv','FRF_CS05-SBE26_Waves_ALL.csv']
years=['2016','2017','2018']
months=['01','02','03','04','05','06','07','08','09','10','11','12']

for station in stations:
    print('for station:', station)
    dfw=pd.read_csv(file_path+station[4:8]+'/'+station, index_col='datetime')
    for year in years:
        print('   Processing year:', year)
        for month in months:
            start_date = year+'-'+month+'-01 00:00:00'
            end_date = year+'-'+month+'-31 59:59:59.9999999'

            #print(index, start_date, end_date)
            dfo=dfw.loc[(dfw.index >= start_date) & (dfw.index <= end_date)].copy()
            if dfo.empty:
                print('      No data for', month, year)
            else:
                dfo.to_csv(file_path+station[4:8]+'/'+'FRF_'+station[4:8]+'_SBE26_Waves_'+year+month+'.csv')
        
print('All done!')

for station: FRF_CS01-SBE26_Waves_ALL.csv
   Processing year: 2016
      No data for 01 2016
   Processing year: 2017
      No data for 06 2017
      No data for 07 2017
      No data for 08 2017
      No data for 09 2017
      No data for 10 2017
      No data for 11 2017
   Processing year: 2018
      No data for 02 2018
      No data for 03 2018
      No data for 04 2018
      No data for 05 2018
      No data for 06 2018
      No data for 07 2018
      No data for 08 2018
      No data for 09 2018
      No data for 10 2018
      No data for 11 2018
      No data for 12 2018
for station: FRF_CS02-SBE26_Waves_ALL.csv
   Processing year: 2016
      No data for 01 2016
      No data for 02 2016
   Processing year: 2017
      No data for 11 2017
      No data for 12 2017
   Processing year: 2018
      No data for 01 2018
      No data for 02 2018
      No data for 03 2018
      No data for 04 2018
      No data for 05 2018
      No data for 06 2018
      No data for 07 2018
      No dat